# Time Series Cross Validation TBATS 

**Trigonometric seasonality, Box-Cox transformation, ARIMA errors, Trend and Seasonal components**

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#forecast error metrics
from forecast_tools.metrics import (mean_absolute_scaled_error, 
                                    root_mean_squared_error,
                                    symmetric_mean_absolute_percentage_error)

from tbats import TBATS, BATS

import time

import warnings
warnings.filterwarnings('ignore')

In [4]:
from amb_forecast.feature_engineering import featurize_time_series

# Data Input

The constants `TOP_LEVEL`, `STAGE`, `REGION`,`TRUST` and `METHOD` are used to control data selection and the directory for outputting results.  

> Output file is `f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv'.csv`.  where metric will be smape, rmse, mase, coverage_80 and coverage_95. Note: `REGION`: is also used to select the correct data from the input dataframe.

In [44]:
TOP_LEVEL = '../../../results/model_selection'
STAGE = 'stage1'
REGION = 'Trust'
METHOD = 'tbats'

FILE_NAME = 'Daily_Responses_5_Years_2019_full.csv'

#split training and test data.
TEST_SPLIT_DATE = '2019-01-01'

#second subdivide: train and val
VAL_SPLIT_DATE = '2017-07-01'

#discard data after 2020 due to coronavirus
#this is the subject of a seperate study.
DISCARD_DATE = '2020-01-01'

In [6]:
#read in path
path = f'../../../data/{FILE_NAME}'

In [7]:
def pre_process_daily_data(path, index_col, by_col, 
                           values, dayfirst=False):
    '''
    Daily data is stored in long format.  Read in 
    and pivot to wide format so that there is a single 
    colmumn for each regions time series.
    '''
    df = pd.read_csv(path, index_col=index_col, parse_dates=True, 
                     dayfirst=dayfirst)
    df.columns = map(str.lower, df.columns)
    df.index.rename(str(df.index.name).lower(), inplace=True)
    
    clean_table = pd.pivot_table(df, values=values.lower(), 
                                 index=[index_col.lower()],
                                 columns=[by_col.lower()], aggfunc=np.sum)
    
    clean_table.index.freq = 'D'
    
    return clean_table

In [8]:
clean = pre_process_daily_data(path, 'Actual_dt', 'ORA', 'Actual_Value', 
                               dayfirst=False)
clean.head()

ora,BNSSG,Cornwall,Devon,Dorset,Gloucestershire,OOA,Somerset,Trust,Wiltshire
actual_dt,,,,,,,,,
2013-12-30,415.0,220.0,502.0,336.0,129.0,NaN,183.0,2042.0,255.0
2013-12-31,420.0,236.0,468.0,302.0,128.0,NaN,180.0,1996.0,260.0
2014-01-01,549.0,341.0,566.0,392.0,157.0,NaN,213.0,2570.0,351.0
2014-01-02,450.0,218.0,499.0,301.0,115.0,NaN,167.0,2013.0,258.0
2014-01-03,419.0,229.0,503.0,304.0,135.0,NaN,195.0,2056.0,269.0


## Train Test Split

In [9]:
def ts_train_test_split(data, split_date):
    '''
    Split time series into training and test data
    
    Parameters:
    -------
    data - pd.DataFrame - time series data.  Index expected as datatimeindex
    split_date - the date on which to split the time series
    
    Returns:
    --------
    tuple (len=2) 
    0. pandas.DataFrame - training dataset
    1. pandas.DataFrame - test dataset
    '''
    train = data.loc[data.index < split_date]
    test = data.loc[data.index >= split_date]
    return train, test

In [10]:
train, test = ts_train_test_split(clean, split_date=TEST_SPLIT_DATE)

#exclude data after 2020 due to coronavirus.
test, discard = ts_train_test_split(test, split_date=DISCARD_DATE)

#train split into train and validation
train, val = ts_train_test_split(train, split_date=VAL_SPLIT_DATE)

In [11]:
train.shape

(1279, 9)

In [12]:
val.shape

(549, 9)

## Time Series Cross Validation

`time_series_cv` implements rolling forecast origin cross validation for time series.  
It does not calculate forecast error, but instead returns the predictions, pred intervals and actuals in an array that can be passed to any forecast error function. (this is for efficiency and allows additional metrics to be calculated if needed).

In [13]:
def time_series_cv(model, y_train, y_val, horizons, cl=0.80, 
                   step=1):
    '''
    Time series cross validation across multiple horizons for a single tbats 
    model.

    Incrementally adds additional training data to the model and tests
    across a provided list of forecast horizons. Note that function tests a
    model only against complete validation sets.  E.g. if horizon = 15 and 
    len(val) = 12 then no testing is done.  In the case of multiple horizons
    e.g. [7, 14, 28] then the function will use the maximum forecast horizon
    to calculate the number of iterations i.e if len(val) = 365 and step = 1
    then no. iterations = len(val) - max(horizon) = 365 - 28 = 337.
    
    Parameters:
    --------
    model: fitted tbats model

    y_train: np.array
        vector of y training data
        
    X_train: array-like
        matrix of X training data

    y_val: np.array
        vector of y validation data
        
    X_val: array-like
        matrix of X validation data

    horizon: list of ints, 
        forecast horizon e.g. [7, 14, 28] days

    step : int, optional (default=1)
        step taken in cross validation 
        e.g. 1 in next cross validation training data includes next point 
        from the validation set.
        e.g. 7 in the next cross validation training data includes next 7 points
        
            
    Returns:
    -------
    tuple: np.ndarray, np.ndarray, np.ndarray    
        predictions, validation_set, prediction_intervals
    '''
    cv_preds = [] #mean forecast
    cv_actuals = [] # actuals 
    cv_pis = [] #prediction intervals
    split = 0

    print('split => ', end="")
    
    for i in range(0, len(val) - max(horizons) + 1, step):
        split += 1
        print(f'{split}, ', end="")
        
        if i > 0:
            #create new training y value and exogenous variables
            y_train_cv = np.concatenate([y_train.iloc[:], 
                                         y_val.iloc[:i]], axis=0)   

            #refits tbats model - it does not change model parameters
            model.fit(y_train_cv)
                
        #max forecast horizon
        horizon=len(y_val[i:i+max(horizons)])
                
        #predict the maximum horizon        
        preds, pis = fitted_model.forecast(steps=horizon, 
                                           confidence_level=cl)

        
        cv_h_preds = []
        cv_test = []
        cv_h_pis = []
        
        for h in horizons:
            #store the h-step prediction
            cv_h_preds.append(preds[:h])
            #store the h-step actual value
            cv_test.append(y_val.iloc[i:i+h])    
            
            #pis is a dictionary 'lower_bound' and 'upper_bound'
            lower = pis['lower_bound'][:h]
            upper = pis['upper_bound'][:h]
            cv_h_pis.append(np.vstack([lower, upper]).T)
                     
        cv_preds.append(cv_h_preds)
        cv_actuals.append(cv_test)
        cv_pis.append(cv_h_pis)
        
    print('done.\n')        
    return cv_preds, cv_actuals, cv_pis

## Custom functions for calculating CV scores for point predictions and coverage.

These functions have been written to work with the output of `time_series_cv`

In [35]:
def split_cv_error(cv_preds, cv_test, error_func):
    '''
    Forecast error in the current split
    
    Params:
    -----
    cv_preds, np.array
        Split predictions
        
    
    cv_test: np.array
        acutal ground truth observations
        
    error_func: object
        function with signature (y_true, y_preds)
        
    Returns:
    -------
        np.ndarray
            cross validation errors for split
    '''
    n_splits = len(cv_preds)
    cv_errors = []
    
    for split in range(n_splits):
        pred_error = error_func(cv_test[split], cv_preds[split])
        cv_errors.append(pred_error)
        
    return np.array(cv_errors)

def forecast_errors_cv(cv_preds, cv_test, error_func):
    '''
    Forecast errors by forecast horizon
    
    Params:
    ------
    cv_preds: np.ndarray
        Array of arrays.  Each array is of size h representing
        the forecast horizon specified.
        
    cv_test: np.ndarray
        Array of arrays.  Each array is of size h representing
        the forecast horizon specified.
        
    error_func: object
        function with signature (y_true, y_preds)
        
    Returns:
    -------
    np.ndarray
        
    '''
    cv_test = np.array(cv_test)
    cv_preds = np.array(cv_preds)
    n_horizons = len(cv_test)    
    
    horizon_errors = []
    for h in range(n_horizons):
        split_errors = split_cv_error(cv_preds[h], cv_test[h], error_func)
        horizon_errors.append(split_errors)

    return np.array(horizon_errors)

def split_coverage(cv_test, cv_intervals):
    n_splits = len(cv_test)
    cv_errors = []
        
    for split in range(n_splits):
        val = np.asarray(cv_test[split])
        lower = cv_intervals[split].T[0]
        upper = cv_intervals[split].T[1]
        
        coverage = len(np.where((val > lower) & (val < upper))[0])
        coverage = coverage / len(val)
        
        cv_errors.append(coverage)
        
    return np.array(cv_errors)
    
    
def prediction_int_coverage_cv(cv_test, cv_intervals):
    cv_test = np.array(cv_test)
    cv_intervals = np.array(cv_intervals)
    n_horizons = len(cv_test)    
    
    horizon_coverage = []
    for h in range(n_horizons):
        split_coverages = split_coverage(cv_test[h], cv_intervals[h])
        horizon_coverage.append(split_coverages)

    return np.array(horizon_coverage)  

In [36]:
def split_cv_error_scaled(cv_preds, cv_test, y_train):
    n_splits = len(cv_preds)
    cv_errors = []
    
    for split in range(n_splits):
        pred_error = mean_absolute_scaled_error(cv_test[split], cv_preds[split], 
                                                y_train, period=7)
        
        cv_errors.append(pred_error)
        
    return np.array(cv_errors)

def forecast_errors_cv_scaled(cv_preds, cv_test, y_train):
    cv_test = np.array(cv_test)
    cv_preds = np.array(cv_preds)
    n_horizons = len(cv_test)    
    
    horizon_errors = []
    for h in range(n_horizons):
        split_errors = split_cv_error_scaled(cv_preds[h], cv_test[h], y_train)
        horizon_errors.append(split_errors)
        
    return np.array(horizon_errors)

# TBATS fitting code

TBATS library provides an autofitting function.  (Parrallel fitting seems v.slow as noted on GitHub)

In [25]:
def get_tbats_fitted(y_train):
    '''
    Return an automatically selected TBATS model.
    '''
    print('Selecting model...', end=' ')
    
    #monitor time taken
    t0 = time.time()
    
    # Create estimator
    estimator = TBATS(
        seasonal_periods=[7, 365.25],
        use_arma_errors=None,  
        use_box_cox=None,
        n_jobs=1
    )
    
    fitted_model = estimator.fit(y_train)
    
    t1 = time.time()
    diff = t1 - t0
    print(f'Time taken: {(diff/60)}')
    return fitted_model

In [26]:
fitted_model = get_tbats_fitted(train['Trust'])

Selecting model... Time taken: 0.9235448837280273


In [27]:
# Time series analysis
print(fitted_model.aic)

# Reading model parameters
print(fitted_model.params.alpha)
print(fitted_model.params.beta)
print(fitted_model.params.x0)
print(fitted_model.params.components.use_box_cox)
print(fitted_model.params.components.seasonal_harmonics)

20258.57405232195
0.3509404115540468
None
[ 8.90428788e+00  1.25886028e-02 -1.58621366e-02  4.38648047e-03
 -6.51350239e-02 -8.01599634e-03  6.59174861e-03  1.39231962e-02
 -6.20978421e-03]
True
[3 1]


# Run Cross Validation

This is run twices once each for 80 and 95% prediction intervals.

In [41]:
horizons = [7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 365]
print('cross-validation...')
#results with 80% CI
results = time_series_cv(fitted_model,
                         y_train=train[REGION], 
                         y_val=val[REGION], 
                         horizons=horizons, 
                         step=7,
                         cl=0.80)

cross-validation...
split => 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, done.



In [42]:
cv_preds, cv_test, cv_intervals = results

In [43]:
#CV point predictions smape
cv_errors = forecast_errors_cv(cv_preds, cv_test, 
                               symmetric_mean_absolute_percentage_error)
df = pd.DataFrame(cv_errors)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,3.118396,3.478986,3.705974,3.851662,4.024748,4.185049,4.288618,4.358313,4.438718,4.492199,4.524415,4.558134,5.037421
std,2.040038,2.163380,2.355032,2.368982,2.407840,2.437763,2.431798,2.386344,2.333106,2.289512,2.208013,2.128884,1.766803
min,1.081646,1.376828,1.909551,1.889062,1.962738,2.297724,2.244062,2.249422,2.337106,2.400585,2.533608,2.757034,3.244951
25%,2.051708,2.328081,2.691733,2.800568,2.855692,3.056741,2.974578,3.182439,3.198898,3.456824,3.557360,3.672779,3.498451
50%,2.304348,2.793041,2.991741,3.180568,3.263182,3.560579,3.606964,3.735272,3.826104,3.898857,3.955488,3.917913,4.651977
75%,3.536241,3.746939,3.478040,3.936475,4.344410,4.417290,4.800781,4.688025,4.532958,4.471733,4.499636,4.797826,5.800768
max,11.280364,11.628651,13.202335,13.830877,14.187979,14.515391,14.628611,14.523720,14.230642,14.389750,14.256715,13.796030,9.712243


In [45]:
#output sMAPE results to file
metric = 'smape'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-tbats_smape.csv


In [46]:
#CV point predictions rmse
cv_errors = forecast_errors_cv(cv_preds, cv_test, root_mean_squared_error)
df = pd.DataFrame(cv_errors)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,82.983805,94.064381,102.129947,106.985513,112.475849,117.285268,120.609777,123.020672,125.672639,127.885187,129.604117,131.428693,141.956597
std,52.418932,57.130835,60.694500,60.507306,60.474579,60.294821,59.590248,58.784411,57.526844,55.815658,53.105494,51.092823,39.446029
min,31.227766,40.711768,52.348866,53.953017,54.742817,65.101428,62.974008,62.817504,65.497106,66.043871,68.954013,74.923231,102.709451
25%,54.540088,61.457877,72.113493,74.890680,76.549440,82.061168,82.449512,86.932858,87.740078,94.401603,98.024229,101.270189,108.829968
50%,67.496017,76.588213,80.220964,84.038570,90.437406,98.243727,99.964287,98.760449,108.163176,113.686750,120.813285,125.102942,131.109700
75%,89.391156,101.446400,101.221587,106.578390,123.026426,137.555095,146.943049,147.006988,141.184462,141.372404,138.854937,145.124738,158.341746
max,270.024504,281.117429,318.655096,332.742596,339.360831,345.705562,347.559183,345.822898,339.763888,342.757890,339.444618,330.763551,251.993047


In [47]:
#output rmse
metric = 'rmse'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-tbats_rmse.csv


In [48]:
#mase
cv_errors = forecast_errors_cv_scaled(cv_preds, cv_test, train[REGION])
df = pd.DataFrame(cv_errors)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,0.846507,0.947277,1.011246,1.052642,1.101001,1.145930,1.175216,1.194743,1.216950,1.231791,1.241157,1.250913,1.366064
std,0.593904,0.629908,0.676986,0.678552,0.687344,0.693534,0.690023,0.676473,0.660701,0.645697,0.621569,0.599348,0.490658
min,0.306426,0.395045,0.513961,0.509636,0.531234,0.618560,0.604709,0.607521,0.631662,0.649666,0.686952,0.750497,0.885453
25%,0.546546,0.611534,0.695993,0.744093,0.753146,0.781787,0.773584,0.836882,0.873508,0.924679,0.962729,0.984586,0.951883
50%,0.614936,0.749045,0.802970,0.820078,0.850225,0.920143,0.953109,1.022520,1.069379,1.074298,1.121330,1.062427,1.273287
75%,0.915047,0.994084,0.914467,1.037152,1.226360,1.249803,1.357895,1.307838,1.268921,1.258400,1.276682,1.346927,1.569261
max,3.165265,3.251775,3.667990,3.834274,3.926229,4.008726,4.039435,4.011978,3.930078,3.970246,3.935699,3.812895,2.723250


In [49]:
#output rmse
metric = 'mase'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-tbats_mase.csv


In [50]:
#80% pi coverage
cv_coverage = prediction_int_coverage_cv(cv_test, cv_intervals)
df = pd.DataFrame(cv_coverage)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,0.841270,0.846561,0.858907,0.871693,0.877249,0.886243,0.891912,0.896825,0.902998,0.906349,0.911977,0.917108,0.976966
std,0.225297,0.229072,0.221188,0.210185,0.208578,0.202498,0.198011,0.191005,0.179006,0.178707,0.169074,0.155727,0.034972
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017857,0.079365,0.071429,0.116883,0.190476,0.813699
25%,0.785714,0.821429,0.857143,0.875000,0.871429,0.880952,0.897959,0.901786,0.904762,0.914286,0.922078,0.928571,0.982192
50%,0.857143,0.928571,0.952381,0.928571,0.942857,0.952381,0.959184,0.946429,0.952381,0.957143,0.948052,0.952381,0.986301
75%,1.000000,1.000000,0.952381,0.964286,0.971429,0.976190,0.979592,0.973214,0.976190,0.978571,0.974026,0.976190,0.989041
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.994521


In [51]:
#output 80% PI coverage
metric = 'coverage_80'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-tbats_coverage_80.csv


In [52]:
#90% PIs
fitted_model = get_tbats_fitted(train[REGION])
horizons = [7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 365]
print('cross-validation...')
#results with 80% CI
results = time_series_cv(fitted_model,
                         y_train=train[REGION], 
                         y_val=val[REGION], 
                         horizons=horizons, 
                         step=7,
                         cl=0.95)

Selecting model... Time taken: 0.91767524878184
cross-validation...
split => 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, done.



In [54]:
#95% PIs
cv_preds, cv_test, cv_intervals = results
cv_coverage = prediction_int_coverage_cv(cv_actuals, cv_intervals)
df = pd.DataFrame(cv_coverage)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,0.973545,0.968254,0.973545,0.977513,0.979894,0.982363,0.983371,0.984127,0.985303,0.986243,0.987013,0.987654,0.995941
std,0.056550,0.080064,0.062415,0.047657,0.038671,0.032748,0.027751,0.024431,0.021979,0.019604,0.017646,0.016000,0.003074
min,0.857143,0.642857,0.714286,0.785714,0.828571,0.857143,0.877551,0.892857,0.904762,0.914286,0.922078,0.928571,0.983562
25%,1.000000,1.000000,0.976190,0.964286,0.971429,0.976190,0.979592,0.973214,0.968254,0.971429,0.974026,0.976190,0.994521
50%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.987013,0.988095,0.997260
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.997260
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [56]:
#output 95% PI coverage
metric = 'coverage_95'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-tbats_coverage_95.csv


# End